<a href="https://colab.research.google.com/github/trushpatel97/ColabResearch/blob/master/EyeDectection_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential
import os

Using TensorFlow backend.


In [ ]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [ ]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

TPU not found


Below code does not work


```
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
```

Mount the Google Drive to obtain dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_path = "/content/drive/My Drive/Research/train"
test_path = "/content/drive/My Drive/Research/test"



Read the image files and split them into train, test and validation

In [ ]:
imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.20)
train_batches = imgdata.flow_from_directory(train_path,class_mode='categorical', subset='training', target_size=(300, 196) ,
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 64)
test_batches = imgdata.flow_from_directory(test_path, target_size=(300, 196) ,class_mode='categorical',
                                           classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 64)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(300, 196) ,class_mode='categorical',
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 64)


Found 6893 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.


In [ ]:

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)


Running on TPU  ['10.67.180.234:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.67.180.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.67.180.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
model = Sequential()
# add the convolutional layer
# filters, size of filters,input_shape,activation_function
model.add(Conv2D(64,(5,5),activation='relu',input_shape=(300,196,1)))

# pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# place a dropout layer
model.add(Dropout(0.25))

# add another convolutional layer
model.add(Conv2D(128,(5,5),activation='relu'))

# pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten the image to 1 dimensional array
model.add(Flatten())

# add a dense layer : amount of nodes, activation
model.add(Dense(256,activation='relu'))

# place a dropout layer
# 0.5 drop out rate is recommended, half input nodes will be dropped at each update
model.add(Dropout(0.5))

# defining the ouput layer of our network
model.add(Dense(4,activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 296, 192, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 96, 32)       0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 148, 96, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 144, 92, 64)       51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 72, 46, 64)        0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 211968)            0         
_________________________________________________________________
dense_53 (Dense)             (None, 256)             

In [ ]:
lr = [0.0005]
for i in lr:
  print("Learning Rate: " + str(i))
  with strategy.scope():
    model = model_create()
    model.compile(Adam(learning_rate=i), loss='categorical_crossentropy', 
              metrics=['accuracy'])
    
  model.fit(train_batches, steps_per_epoch = 128, epochs = 10, 
            validation_data = valid_batches, validation_steps = 32, 
            class_weight = class_weights, shuffle = True)

Learning Rate: 0.0005
Epoch 1/10
  1/128 [..............................] - ETA: 41:27 - loss: 1.6333 - accuracy: 0.0781

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 9 could not be retrieved. It could be because a worker has died.
  UserWarning)


128/128 [==============================] - 2588s 20s/step - loss: 0.6684 - accuracy: 0.7555 - val_loss: 0.5717 - val_accuracy: 0.8247
Epoch 2/10
128/128 [==============================] - 2384s 19s/step - loss: 0.4452 - accuracy: 0.8392 - val_loss: 0.4434 - val_accuracy: 0.8452
Epoch 3/10
128/128 [==============================] - 2320s 18s/step - loss: 0.4096 - accuracy: 0.8500 - val_loss: 0.7039 - val_accuracy: 0.8184
Epoch 4/10
128/128 [==============================] - 2399s 19s/step - loss: 0.4037 - accuracy: 0.8551 - val_loss: 0.4385 - val_accuracy: 0.8770
Epoch 5/10
128/128 [==============================] - 2340s 18s/step - loss: 0.3786 - accuracy: 0.8646 - val_loss: 0.2608 - val_accuracy: 0.8564
Epoch 6/10
127/128 [============================>.] - ETA: 15s - loss: 0.3487 - accuracy: 0.8757

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 179 could not be retrieved. It could be because a worker has died.
  UserWarning)


128/128 [==============================] - 2349s 18s/step - loss: 0.3490 - accuracy: 0.8752 - val_loss: 0.4768 - val_accuracy: 0.8613
Epoch 7/10
 55/128 [===========>..................] - ETA: 16:29 - loss: 0.3484 - accuracy: 0.8750

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 864 could not be retrieved. It could be because a worker has died.
  UserWarning)


128/128 [==============================] - 2311s 18s/step - loss: 0.3411 - accuracy: 0.8779 - val_loss: 0.3297 - val_accuracy: 0.8628
Epoch 8/10
128/128 [==============================] - 2312s 18s/step - loss: 0.3602 - accuracy: 0.8718 - val_loss: 0.3133 - val_accuracy: 0.8687
Epoch 9/10
128/128 [==============================] - 316s 2s/step - loss: 0.3175 - accuracy: 0.8870 - val_loss: 0.3427 - val_accuracy: 0.8407
Epoch 10/10
128/128 [==============================] - 194s 2s/step - loss: 0.2990 - accuracy: 0.8942 - val_loss: 0.3195 - val_accuracy: 0.8828


In [ ]:
model.save("Made_Model.h5")

NameError: ignored